<a href="https://colab.research.google.com/github/jamccauley/rfi-detection-project/blob/main/CS_677_Course_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

README - R-NET, LSTM, CNN-LSTM, & Gaussian Process Model Implementations
 
RFI Detection in Radio Astronomy Data  
Morgan Dameron & John McCauley

NOTE: The code in this notebook was written by John McCauley

This document along with the provided code and data describes how to run the experiements used in this project. A list of all required python packages/modules is included at the end of the notebook. 

I have tried to format filepath references to work as written, but I cannot confirm this. As such to properly load the sample data, some filename references in the code may need changed. Those lines are marked with comments indicating such in the following code blocks, just in case here are some instructions on changing the file location references:

Directly below the comment "#Load PSD Data and Masks (s_16 and TONE)" in each of the four main code blocks there are four lines of code in which filepaths are referenced. Those lines are:

"s16_filepath = "/Data/s_16bit.npy"
s16flag_filepath = "/Data/s_16bit_flags.npy"
tone_filepath = "/Data/TONE_Data.npy"
toneflag_filepath = "/Data/TONE_data_flags.npy""


Please change the file path references here if the current reference is incorrect.

---



Section 1. R-Net

---


The first block of code instantiates the R-Net5 model as described in the original publication. Please pay special attention to the comments in the code about the input shape of the network, depending on which data set is to be used, the input shape must be adjusted. The code block then loads in the sample data, performs the necessary transformations, splits the data into train and test splits, and begins a training session on the data. Once training is complete, the model is saved, predicted values for the test set are calculated, and classification metrics are calculated.

Currently the model will train and test on the sample TONE data, to change this please replace the line containing the input layer of R-Net to

"input = keras.Input(shape=(4096, 512, 1))"

As well as replace the lines below these comments

"#concatenate data into overall data matrix  
"#change which vars are in the concatenate block to train/test on a different dataset

to this - 

"psdData = np.concatenate((psd0Blocks, psd1Blocks),axis=0)  
psdLabels = np.concatenate((maskPsd0Blocks, maskPsd1Blocks), axis=0) 
"


In [ ]:

##### RFI DETECTION IN RADIO ASTRONOMY DATA #####
##### Morgan Dameron & John McCauley #####

# R-Net Implementation - Architecture for R-Net5
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

input = keras.Input(shape=(1024, 512, 1)) #Change input shape depending on which data being used. s_16 = (4096, 512, 1), TONE = (1024, 512, 1)
xp = layers.Conv2D(filters=12,kernel_size=5,strides=(1, 1),padding='same')(input)
x = layers.BatchNormalization()(xp)
x = layers.ReLU()(x)
x1 = layers.Conv2D(filters=12,kernel_size=5,strides=(1, 1),padding='same')(x)
x1 = layers.BatchNormalization()(x1)
x1 = layers.ReLU()(x1)
x2 = layers.Conv2D(filters=12,kernel_size=5,strides=(1, 1),padding='same')(x1)
x2 = layers.BatchNormalization()(x2)
x2 = layers.ReLU()(x2)
x3 = x2+xp
x4 = layers.Conv2D(filters=12,kernel_size=5,strides=(1, 1),padding='same')(x3)
x4 = layers.BatchNormalization()(x4)
x4 = layers.ReLU()(x4)
x5 = layers.Conv2D(filters=1,kernel_size=5,strides=(1, 1),padding='same')(x4)
x5 = layers.BatchNormalization()(x5)
x5 = layers.ReLU()(x5)

x_out = layers.Conv2D(filters=1,kernel_size=5,strides=(1, 1),padding='same',
            activation=tf.nn.relu)(x5)

model = keras.Model(inputs=input,outputs=x_out)

model.summary()
keras.utils.plot_model(model)

model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.RMSprop(learning_rate=0.001),
    metrics=["accuracy"],
)

# R-Net Implementation - Load all data
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


#Load PSD Data and Masks (s_16 and TONE)
s16_filepath = "/Data/s_16bit.npy"
s16flag_filepath = "/Data/s_16bit_flags.npy"
tone_filepath = "/Data/TONE_Data.npy"
toneflag_filepath = "/Data/TONE_data_flags.npy"

psd = np.load(s16_filepath) #load the .npy file, filepath may need changed

psd0Blocks = np.stack(np.split(psd[:,0:3072,0],6,axis=1)) #split into 127 blocks along the time axis for each polarity
psd1Blocks = np.stack(np.split(psd[:,0:3072,1],6,axis=1))

psd0Blocks = np.array(psd0Blocks, dtype = np.float32)
psd1Blocks = np.array(psd1Blocks, dtype = np.float32)

maskPsd = np.load(s16flag_filepath) #load the SK g.t., filepath may need changed


Mask0 = np.swapaxes(maskPsd[0:3072,:,0],0,1)
Mask1 = np.swapaxes(maskPsd[0:3072,:,1],0,1)

#tile ground truth labels to match input size
maskLab0 = []
maskLab1 = []
for i in range(np.shape(Mask0)[0]):
  x1 = Mask0[i,:]
  y1 = np.tile(x1, (512, 1))
  x2 = Mask1[i,:]
  y2 = np.tile(x2, (512, 1))
  maskLab0.append(np.transpose(y1))
  maskLab1.append(np.transpose(y2))

maskPsd0Blocks = np.array(maskLab0,dtype=np.float32)
maskPsd1Blocks = np.array(maskLab1,dtype=np.float32)

tonePsd = np.load(tone_filepath) #load TONE data, filepath may need changed
toneMask = np.load(toneflag_filepath) #load TONE SK g.t., filepath may need changed

tonePsd0Blocks = np.stack(np.split(tonePsd[:,0:3072,0],6,axis=1)) #split into 127 blocks along the time axis for each polarity
tonePsd1Blocks = np.stack(np.split(tonePsd[:,0:3072,1],6,axis=1))

tonePsd0Blocks = np.array(tonePsd0Blocks, dtype = np.float32)
tonePsd1Blocks = np.array(tonePsd1Blocks, dtype = np.float32)

toneMask0 = np.swapaxes(toneMask[0:3072,:,0],0,1)
toneMask1 = np.swapaxes(toneMask[0:3072,:,1],0,1)

#tile ground truth labels to match input size
toneLab0 = []
toneLab1 = []
for i in range(np.shape(toneMask0)[0]):
  x1 = toneMask0[i,:]
  y1 = np.tile(x1, (512, 1))
  x2 = toneMask1[i,:]
  y2 = np.tile(x2, (512, 1))
  toneLab0.append(np.transpose(y1))
  toneLab1.append(np.transpose(y2))

toneMask0Blocks = np.array(toneLab0,dtype=np.float32)
toneMask1Blocks = np.array(toneLab1,dtype=np.float32)

#concatenate data into overall data matrix
#change which vars are in the concatenate block to train/test on a different dataset
psdData = np.concatenate((tonePsd0Blocks, tonePsd1Blocks),axis=0) # tonePsd0Blocks, tonePsd1Blocks psd0Blocks, psd1Blocks
psdLabels = np.concatenate((toneMask0Blocks, toneMask1Blocks), axis=0) # toneMask0Blocks, toneMask1Blocks maskPsd0Blocks, maskPsd1Blocks

del psd, maskPsd, Mask0, Mask1, toneMask0Blocks, toneMask1Blocks, tonePsd, toneMask, maskLab1, toneLab0 #delete all old variables to save on RAM
del psd0Blocks, psd1Blocks, maskPsd0Blocks, maskPsd1Blocks, toneMask0, toneMask1, maskLab0, toneLab1

#normalize all samples
normPsd = []
for i in range(np.shape(psdData)[0]):
  sample = psdData[i,:,:]
  sample = sample - np.min(sample)
  sample = sample / np.max(sample)
  normPsd.append(sample)

#split into train test
X_train, X_test, Y_train, Y_test = train_test_split(normPsd, psdLabels, test_size = 0.2, random_state=1)

X_train = np.expand_dims(X_train, axis=3)
X_test = np.expand_dims(X_test, axis=3)
Y_train = np.expand_dims(Y_train, axis=3)
Y_test = np.expand_dims(Y_test, axis=3)

##### Train & Test R-Net5 #####
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#model = keras.models.load_model("/content/drive/Shareddrives/CS 677/Project/R-Net_s16sk/") #load an old model to train further

history = model.fit(X_train, Y_train, batch_size=10, epochs=15, validation_split=0.2) #Change params here for the situation (i.e. smaller batch size for s_16 data, smaller number of epochs for further training on old model) 

model.save("/") #make sure to change model save location for a new model

preds = model.predict(X_test,batch_size=10) #get predicted values from trained model

print(np.shape(preds))

##### Get classification metrics from predicted vals #####
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,auc,roc_curve

#Create Binary Mask from Predicted Values for Each Sample
maskedPreds = []
for i in range(np.shape(preds)[0]):
  toMask = preds[i,:,:,0]
  thresh = 0.6 #fine tune threshold for best results
  mask = np.where(toMask >= thresh, 1, 0) 
  maskedPreds.append(mask)

maskedPreds = np.stack(maskedPreds)

vectTrue = np.ravel(Y_test) #vectorize preds and labels for classification metrics
vectPreds = np.ravel(maskedPreds)

print(classification_report(vectTrue,vectPreds)) #calculate classification metrics

#Combine all time blocks for visualization, both preds and true labels
reMask = []
reTrue = []
for i in range(np.shape(maskedPreds)[0]):
  bb = np.transpose(maskedPreds[i,:,:])
  cc = np.transpose(Y_test[i,:,:,0])
  reMask.append(bb)
  reTrue.append(cc)

reMask = np.concatenate(reMask)
reTrue = np.concatenate(reTrue)

---



Section 2. LSTM

---


The second block of code performs training and testing of the vanilla LSTM model. The code block loads in the sample data, performs the necessary transformations, splits the data into train and test splits, and begins a training session on the data. Once training is complete, the model is saved, predicted values for the test set are calculated, and classification metrics are calculated. 

Currently the model will train and test on the sample TONE data, to change this please replace the lines below the comments

"#concatenate data into overall data matrix"  
"#change which vars are in the concatenate block to train/test on a different dataset"

to this - 

"psdData = np.concatenate((psd0Blocks, psd1Blocks),axis=0)  
psdLabels = np.concatenate((maskPsd0Blocks, maskPsd1Blocks), axis=0) 
"


In [ ]:
##### CS 677 COURSE PROJECT #####
##### RFI DETECTION IN RADIO ASTRONOMY DATA #####
##### Morgan Dameron & John McCauley #####

##### LSTM Classification #####
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#Load PSD Data and Masks (s_16 and TONE)
s16_filepath = "/Data/s_16bit.npy"
s16flag_filepath = "/Data/s_16bit_flags.npy"
tone_filepath = "/Data/TONE_Data.npy"
toneflag_filepath = "/Data/TONE_data_flags.npy"

psd = np.load(s16_filepath) #load the .npy file, filepath may need changed

psd0Blocks = np.stack(np.split(psd[:,0:3072,0],6,axis=1)) #split into 127 blocks along the time axis for each polarity
psd1Blocks = np.stack(np.split(psd[:,0:3072,1],6,axis=1))

psd0Blocks = np.array(psd0Blocks, dtype = np.float32)
psd1Blocks = np.array(psd1Blocks, dtype = np.float32)

maskPsd = np.load(s16flag_filepath) #load the SK g.t., filepath may need changed


Mask0 = np.swapaxes(maskPsd[0:3072,:,0],0,1)
Mask1 = np.swapaxes(maskPsd[0:3072,:,1],0,1)

#tile ground truth labels to match input size
maskLab0 = []
maskLab1 = []
for i in range(np.shape(Mask0)[0]):
  x1 = Mask0[i,:]
  y1 = np.tile(x1, (512, 1))
  x2 = Mask1[i,:]
  y2 = np.tile(x2, (512, 1))
  maskLab0.append(np.transpose(y1))
  maskLab1.append(np.transpose(y2))

maskPsd0Blocks = np.array(maskLab0,dtype=np.float32)
maskPsd1Blocks = np.array(maskLab1,dtype=np.float32)

tonePsd = np.load(tone_filepath) #load TONE data, filepath may need changed
toneMask = np.load(toneflag_filepath) #load TONE SK g.t., filepath may need changed

tonePsd0Blocks = np.stack(np.split(tonePsd[:,0:3072,0],6,axis=1)) #split into 127 blocks along the time axis for each polarity
tonePsd1Blocks = np.stack(np.split(tonePsd[:,0:3072,1],6,axis=1))

tonePsd0Blocks = np.array(tonePsd0Blocks, dtype = np.float32)
tonePsd1Blocks = np.array(tonePsd1Blocks, dtype = np.float32)

toneMask0 = np.swapaxes(toneMask[0:3072,:,0],0,1)
toneMask1 = np.swapaxes(toneMask[0:3072,:,1],0,1)

#tile ground truth labels to match input size
toneLab0 = []
toneLab1 = []
for i in range(np.shape(toneMask0)[0]):
  x1 = toneMask0[i,:]
  y1 = np.tile(x1, (512, 1))
  x2 = toneMask1[i,:]
  y2 = np.tile(x2, (512, 1))
  toneLab0.append(np.transpose(y1))
  toneLab1.append(np.transpose(y2))

toneMask0Blocks = np.array(toneLab0,dtype=np.float32)
toneMask1Blocks = np.array(toneLab1,dtype=np.float32)

#concatenate data into overall data matrix
#change which vars are in the concatenate block to train/test on a different dataset
psdData = np.concatenate((tonePsd0Blocks, tonePsd1Blocks),axis=0) # tonePsd0Blocks, tonePsd1Blocks psd0Blocks, psd1Blocks
psdLabels = np.concatenate((toneMask0Blocks, toneMask1Blocks), axis=0) # toneMask0Blocks, toneMask1Blocks maskPsd0Blocks, maskPsd1Blocks

del psd, maskPsd, Mask0, Mask1, toneMask0Blocks, toneMask1Blocks, tonePsd, toneMask, maskLab1, toneLab0 #delete all old variables to save on RAM
del psd0Blocks, psd1Blocks, maskPsd0Blocks, maskPsd1Blocks, toneMask0, toneMask1, maskLab0, toneLab1

#normalize all samples
normPsd = []
for i in range(np.shape(psdData)[0]):
  sample = psdData[i,:,:]
  sample = sample - np.min(sample)
  sample = sample / np.max(sample)
  normPsd.append(sample)

normPsd = np.array(normPsd, dtype=np.float32) 
print(np.shape(normPsd))

#reshape samples into univariate time-series
dat = []
lab = []
for i in range(np.shape(normPsd)[0]):
  for j in range(1024):
    samp = normPsd[i,j,:]
    label = psdLabels[i,j,0]
    dat.append(samp)
    lab.append(label)

dat = np.array(dat, dtype=np.float32) 
lab = np.array(lab, dtype=np.float32)

X_train, X_test, Y_train, Y_test = train_test_split(dat, lab, test_size = 0.2, random_state=42)


#Define LSTM Model
input = keras.Input(shape=(512, 1))
x1 = keras.layers.LSTM(50)(input)
x2 = keras.layers.Dropout(0.2)(x1)
x3 = keras.layers.Dense(50, activation='relu')(x2)
x4 = keras.layers.Dense(1, activation='sigmoid')(x3)

model = keras.Model(inputs=input,outputs=x4)
model.summary()
keras.utils.plot_model(model)

model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    metrics=["accuracy"],
)

history = model.fit(X_train,Y_train,batch_size=2048, epochs=50, validation_split=0.2)

model.save("/")

preds = model.predict(X_test,batch_size=512)

print(np.shape(preds))

##### Classification Metrics for LSTM and CNN LSTM #####
from sklearn.metrics import classification_report

thresh = 0.6 #change thresh value to fine tune classification results
binPreds = np.where(preds>thresh,1,0)

print(classification_report(Y_test, binPreds))

---



Section 3. CNN-LSTM

---


The third block of code performs training and testing of the CNN-LSTM model. This code block is structured in the same way as the LSTM block. The block loads in the sample data, performs the necessary transformations, splits the data into train and test splits, and begins a training session on the data. Once training is complete, the model is saved, predicted values for the test set are calculated, and classification metrics are calculated.

Currently the model will train and test on the sample TONE data, to change this please replace the lines below the comments

"#concatenate data into overall data matrix"  
"#change which vars are in the concatenate block to train/test on a different dataset"

to this - 

"psdData = np.concatenate((psd0Blocks, psd1Blocks),axis=0)  
psdLabels = np.concatenate((maskPsd0Blocks, maskPsd1Blocks), axis=0) 
"

In [ ]:
##### CS 677 COURSE PROJECT #####
##### RFI DETECTION IN RADIO ASTRONOMY DATA #####
##### Morgan Dameron & John McCauley #####

##### CNN-LSTM Implementation #####
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

#Load PSD Data and Masks (s_16 and TONE)
s16_filepath = "/Data/s_16bit.npy"
s16flag_filepath = "/Data/s_16bit_flags.npy"
tone_filepath = "/Data/TONE_Data.npy"
toneflag_filepath = "/Data/TONE_data_flags.npy"

psd = np.load(s16_filepath) #load the .npy file, filepath may need changed

psd0Blocks = np.stack(np.split(psd[:,0:3072,0],6,axis=1)) #split into 127 blocks along the time axis for each polarity
psd1Blocks = np.stack(np.split(psd[:,0:3072,1],6,axis=1))

psd0Blocks = np.array(psd0Blocks, dtype = np.float32)
psd1Blocks = np.array(psd1Blocks, dtype = np.float32)

maskPsd = np.load(s16flag_filepath) #load the SK g.t., filepath may need changed


Mask0 = np.swapaxes(maskPsd[0:3072,:,0],0,1)
Mask1 = np.swapaxes(maskPsd[0:3072,:,1],0,1)

#tile ground truth labels to match input size
maskLab0 = []
maskLab1 = []
for i in range(np.shape(Mask0)[0]):
  x1 = Mask0[i,:]
  y1 = np.tile(x1, (512, 1))
  x2 = Mask1[i,:]
  y2 = np.tile(x2, (512, 1))
  maskLab0.append(np.transpose(y1))
  maskLab1.append(np.transpose(y2))

maskPsd0Blocks = np.array(maskLab0,dtype=np.float32)
maskPsd1Blocks = np.array(maskLab1,dtype=np.float32)

tonePsd = np.load(tone_filepath) #load TONE data, filepath may need changed
toneMask = np.load(toneflag_filepath) #load TONE SK g.t., filepath may need changed

tonePsd0Blocks = np.stack(np.split(tonePsd[:,0:3072,0],6,axis=1)) #split into 127 blocks along the time axis for each polarity
tonePsd1Blocks = np.stack(np.split(tonePsd[:,0:3072,1],6,axis=1))

tonePsd0Blocks = np.array(tonePsd0Blocks, dtype = np.float32)
tonePsd1Blocks = np.array(tonePsd1Blocks, dtype = np.float32)

toneMask0 = np.swapaxes(toneMask[0:3072,:,0],0,1)
toneMask1 = np.swapaxes(toneMask[0:3072,:,1],0,1)

#tile ground truth labels to match input size
toneLab0 = []
toneLab1 = []
for i in range(np.shape(toneMask0)[0]):
  x1 = toneMask0[i,:]
  y1 = np.tile(x1, (512, 1))
  x2 = toneMask1[i,:]
  y2 = np.tile(x2, (512, 1))
  toneLab0.append(np.transpose(y1))
  toneLab1.append(np.transpose(y2))

toneMask0Blocks = np.array(toneLab0,dtype=np.float32)
toneMask1Blocks = np.array(toneLab1,dtype=np.float32)

#concatenate data into overall data matrix
#change which vars are in the concatenate block to train/test on a different dataset
psdData = np.concatenate((tonePsd0Blocks, tonePsd1Blocks),axis=0) # tonePsd0Blocks, tonePsd1Blocks psd0Blocks, psd1Blocks
psdLabels = np.concatenate((toneMask0Blocks, toneMask1Blocks), axis=0) # toneMask0Blocks, toneMask1Blocks maskPsd0Blocks, maskPsd1Blocks

del psd, maskPsd, Mask0, Mask1, toneMask0Blocks, toneMask1Blocks, tonePsd, toneMask, maskLab1, toneLab0 #delete all old variables to save on RAM
del psd0Blocks, psd1Blocks, maskPsd0Blocks, maskPsd1Blocks, toneMask0, toneMask1, maskLab0, toneLab1

#normalize all samples
normPsd = []
for i in range(np.shape(psdData)[0]):
  sample = psdData[i,:,:]
  sample = sample - np.min(sample)
  sample = sample / np.max(sample)
  normPsd.append(sample)

normPsd = np.array(normPsd, dtype=np.float32) 
print(np.shape(normPsd))

#reshape samples into univariate time-series
dat = []
lab = []
for i in range(np.shape(normPsd)[0]):
  for j in range(1024):
    samp = normPsd[i,j,:]
    label = psdLabels[i,j,0]
    dat.append(samp)
    lab.append(label)

dat = np.array(dat, dtype=np.float32) 
lab = np.array(lab, dtype=np.float32)

X_train, X_test, Y_train, Y_test = train_test_split(dat, lab, test_size = 0.2, random_state=42)

X_train = np.reshape(np.array(X_train, dtype=np.float32),(np.shape(X_train)[0], 4, 128, 1)) 
X_test = np.reshape(np.array(X_test, dtype=np.float32),(np.shape(X_test)[0], 4, 128, 1))
Y_train = np.array(Y_train, dtype=np.float32)
Y_test = np.array(Y_test, dtype=np.float32)


print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(Y_train))
print(np.shape(Y_test))

#Define CNN-LSTM Model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=12, kernel_size=3, activation='relu'), input_shape=(None,128,1)))
model.add(TimeDistributed(Conv1D(filters=12, kernel_size=3, activation='relu')))
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50))
model.add(Dropout(0.3))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)

history = model.fit(X_train,Y_train,batch_size=1024, epochs=50, validation_split=0.2)

model.save("/content/drive/Shareddrives/CS 677/Project/CNN_LSTM_TONE/")

preds = model.predict(X_test,batch_size=512)

print(np.shape(preds))

##### Classification Metrics for LSTM and CNN LSTM #####
from sklearn.metrics import classification_report

thresh = 0.6 #change thresh value to fine tune classification results
binPreds = np.where(preds>thresh,1,0)

print(classification_report(Y_test, binPreds))

---



Section 4. Gaussian Process Classifier

---


The fourth block of code performs training and testing of the Gaussian Process model. This code block is structured in the same way as the previous two blocks. The block loads in the sample data, performs the necessary transformations, splits the data into train and test splits, and begins a training session on the data. Once training is complete, predicted values for the test set are calculated, and classification metrics are calculated.

Currently the model will train and test on the sample TONE data, to change this please replace the lines below the comments

"#concatenate data into overall data matrix"  
"#change which vars are in the concatenate block to train/test on a different dataset"

to this - 

"psdData = np.concatenate((psd0Blocks, psd1Blocks),axis=0)  
psdLabels = np.concatenate((maskPsd0Blocks, maskPsd1Blocks), axis=0) 
"

In [ ]:
##### RFI DETECTION IN RADIO ASTRONOMY DATA #####
##### Morgan Dameron & John McCauley #####

##### Gaussian Process Model #####
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.model_selection import train_test_split

#Load PSD Data and Masks (s_16 and TONE)
s16_filepath = "/Data/s_16bit.npy"
s16flag_filepath = "/Data/s_16bit_flags.npy"
tone_filepath = "/Data/TONE_Data.npy"
toneflag_filepath = "/Data/TONE_data_flags.npy"

psd = np.load(s16_filepath) #load the .npy file, filepath may need changed

psd0Blocks = np.stack(np.split(psd[:,0:3072,0],6,axis=1)) #split into 127 blocks along the time axis for each polarity
psd1Blocks = np.stack(np.split(psd[:,0:3072,1],6,axis=1))

psd0Blocks = np.array(psd0Blocks, dtype = np.float32)
psd1Blocks = np.array(psd1Blocks, dtype = np.float32)

maskPsd = np.load(s16flag_filepath) #load the SK g.t., filepath may need changed


Mask0 = np.swapaxes(maskPsd[0:3072,:,0],0,1)
Mask1 = np.swapaxes(maskPsd[0:3072,:,1],0,1)

#tile ground truth labels to match input size
maskLab0 = []
maskLab1 = []
for i in range(np.shape(Mask0)[0]):
  x1 = Mask0[i,:]
  y1 = np.tile(x1, (512, 1))
  x2 = Mask1[i,:]
  y2 = np.tile(x2, (512, 1))
  maskLab0.append(np.transpose(y1))
  maskLab1.append(np.transpose(y2))

maskPsd0Blocks = np.array(maskLab0,dtype=np.float32)
maskPsd1Blocks = np.array(maskLab1,dtype=np.float32)

tonePsd = np.load(tone_filepath) #load TONE data, filepath may need changed
toneMask = np.load(toneflag_filepath) #load TONE SK g.t., filepath may need changed

tonePsd0Blocks = np.stack(np.split(tonePsd[:,0:3072,0],6,axis=1)) #split into 127 blocks along the time axis for each polarity
tonePsd1Blocks = np.stack(np.split(tonePsd[:,0:3072,1],6,axis=1))

tonePsd0Blocks = np.array(tonePsd0Blocks, dtype = np.float32)
tonePsd1Blocks = np.array(tonePsd1Blocks, dtype = np.float32)

toneMask0 = np.swapaxes(toneMask[0:3072,:,0],0,1)
toneMask1 = np.swapaxes(toneMask[0:3072,:,1],0,1)

#tile ground truth labels to match input size
toneLab0 = []
toneLab1 = []
for i in range(np.shape(toneMask0)[0]):
  x1 = toneMask0[i,:]
  y1 = np.tile(x1, (512, 1))
  x2 = toneMask1[i,:]
  y2 = np.tile(x2, (512, 1))
  toneLab0.append(np.transpose(y1))
  toneLab1.append(np.transpose(y2))

toneMask0Blocks = np.array(toneLab0,dtype=np.float32)
toneMask1Blocks = np.array(toneLab1,dtype=np.float32)

#concatenate data into overall data matrix
#change which vars are in the concatenate block to train/test on a different dataset
psdData = np.concatenate((tonePsd0Blocks, tonePsd1Blocks),axis=0) # tonePsd0Blocks, tonePsd1Blocks psd0Blocks, psd1Blocks
psdLabels = np.concatenate((toneMask0Blocks, toneMask1Blocks), axis=0) # toneMask0Blocks, toneMask1Blocks maskPsd0Blocks, maskPsd1Blocks

del psd, maskPsd, Mask0, Mask1, toneMask0Blocks, toneMask1Blocks, tonePsd, toneMask, maskLab1, toneLab0 #delete all old variables to save on RAM
del psd0Blocks, psd1Blocks, maskPsd0Blocks, maskPsd1Blocks, toneMask0, toneMask1, maskLab0, toneLab1

#normalize all samples
normPsd = []
for i in range(np.shape(psdData)[0]):
  sample = psdData[i,:,:]
  sample = sample - np.min(sample)
  sample = sample / np.max(sample)
  normPsd.append(sample)

normPsd = np.array(normPsd, dtype=np.float32) 

#reshape samples into univariate time-series
dat = []
lab = []
for i in range(np.shape(normPsd)[0]):
  for j in range(1024):
    samp = normPsd[i,j,:]
    label = psdLabels[i,j,0]
    dat.append(samp)
    lab.append(label)

dat = np.array(dat, dtype=np.float32) 
lab = np.array(lab, dtype=np.float32)

X_train, X_test, Y_train, Y_test = train_test_split(dat, lab, test_size = 0.2, random_state=42)

kern = 1.0 * RationalQuadratic(length_scale=1, alpha=1.5)

model = GaussianProcessClassifier(kernel = kern).fit(X_train[0:4096,:], Y_train[0:4096])

preds = model.predict(X_test[0:4096,:])

print(classification_report(Y_test[0:4096],preds))

---



Section 5. Ancillary Methods

---

The following blocks of code are additional methods used to generate the figures and visualizations shown in the presentation and report. The comments at the top of each block explain what the block does. Fine tuning of figure spacing is required for good figure generation, as such these methods may not produce readable figures for every data set/data combination without some edits. The values of 'aspect=' and 'plt.subplots_adjust()' can be edited to produce good visualizations.   
  
For blocks "##### Visualize outputs, one time block #####", "##### Visualize outputs, all time blocks from test set, preds vs true labels #####", and "##### Plot one block of preds #####" - the R-Net code block must be run before running these blocks.

For block "##### Plot Model Loss #####" - This block can be run after training any of the Neural Network architectures (R-Net, LSTM, CNN-LSTM)

In [ ]:
##### Visualize outputs, one time block #####
fig, (ax1, ax2, ax3) = plt.subplots(3,1)
ax1.imshow(preds[0,:,:,0],aspect=0.2)
ax1.set_title('Raw Predictions')
ax1.set_xlabel('Time Bins')
ax1.set_ylabel('Frequency Channels')
ax1.invert_yaxis()
ax2.imshow(maskedPreds[0,:,:],aspect=0.2)
ax2.set_title('Predicted RFI Mask')
ax2.set_xlabel('Time Bins')
ax2.set_ylabel('Frequency Channels')
ax2.invert_yaxis()
ax3.imshow(Y_test[0,:,:,0],aspect=0.2)
ax3.set_title('Ground Truth RFI Mask')
ax3.set_xlabel('Time Bins')
ax3.set_ylabel('Frequency Channels')
ax3.invert_yaxis()
plt.subplots_adjust(top=3.1)
plt.savefig('/content/drive/Shareddrives/CS 677/Project/output_tonernet.png',bbox_inches='tight')

In [ ]:
##### Visualize outputs, all time blocks from test set, preds vs true labels #####
fig, (ax1, ax2) = plt.subplots(2,1)
ax1.imshow(np.transpose(reMask),aspect=5)0
ax1.set_title('Predicted RFI Mask')
ax1.set_xlabel('Time Bins')
ax1.set_ylabel('Frequency Channels')
ax1.invert_yaxis()
plt.subplots_adjust(top=1.01)
ax2.imshow(np.transpose(reTrue),aspect=5)
ax2.set_title('Ground Truth RFI Mask')
ax2.set_xlabel('Time Bins')
ax2.set_ylabel('Frequency Channels')
ax2.invert_yaxis()
plt.savefig('/content/drive/Shareddrives/CS 677/Project/predvgt_s16skrnet.png',bbox_inches='tight')

In [ ]:
##### Plot one block of preds #####
f = plt.figure(1)
plt.imshow(np.flipud(preds[0,:,:,0]),aspect=0.1)
plt.title("Predicted RFI")
plt.xticks([])
plt.yticks([])
plt.savefig('/content/drive/Shareddrives/CS 677/Project/tflabel.png',bbox_inches='tight')

In [ ]:
##### Plot Model Loss #####
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train Loss', 'Validation Loss'], loc='upper right')
plt.savefig('/content/drive/Shareddrives/CS 677/Project/loss_curve_rnets16sk.png')
plt.show()

---



Section 6. Appendix - list of required packages

---

numpy   
matplotlib  
sklearn  
keras  
tensorflow  